In [66]:
import copy
import time
import threading
import pandas as pd
import numpy as np
import more_itertools
import csv
import random
import itertools

pd.options.mode.chained_assignment = None  # default = 'warn'

In [67]:
df = pd.read_csv('Data/master_rules.csv')
df.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,frozenset({'Instant food products'}),frozenset({'soda'}),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,frozenset({'Instant food products'}),frozenset({'whole milk'}),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,frozenset({'UHT-milk'}),frozenset({'whole milk'}),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,frozenset({'abrasive cleaner'}),frozenset({'rolls/buns'}),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,frozenset({'artif. sweetener'}),frozenset({'other vegetables'}),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
5,frozenset({'artif. sweetener'}),frozenset({'whole milk'}),0.007442,0.458045,0.004619,0.620690,1.355085,0.001210,1.428791
6,frozenset({'bathroom cleaner'}),frozenset({'other vegetables'}),0.004362,0.376700,0.002566,0.588235,1.561548,0.000923,1.513729
7,frozenset({'bathroom cleaner'}),frozenset({'whole milk'}),0.004362,0.458045,0.003079,0.705882,1.541078,0.001081,1.842648
8,frozenset({'beef'}),frozenset({'whole milk'}),0.119579,0.458045,0.064152,0.536481,1.171241,0.009379,1.169218
9,frozenset({'bottled beer'}),frozenset({'whole milk'}),0.158840,0.458045,0.085450,0.537964,1.174480,0.012694,1.172973


In [68]:
df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,frozenset({'Instant food products'}),frozenset({'soda'}),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,frozenset({'Instant food products'}),frozenset({'whole milk'}),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,frozenset({'UHT-milk'}),frozenset({'whole milk'}),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,frozenset({'abrasive cleaner'}),frozenset({'rolls/buns'}),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,frozenset({'artif. sweetener'}),frozenset({'other vegetables'}),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24247,"frozenset({'sausage', 'soda', 'whole milk', 'o...",frozenset({'root vegetables'}),0.003849,0.230690,0.002566,0.666667,2.889878,0.001678,2.307929
24248,"frozenset({'sausage', 'whole milk', 'other veg...",frozenset({'soda'}),0.003336,0.313318,0.002566,0.769231,2.455112,0.001521,2.975622
24249,"frozenset({'sausage', 'soda', 'whole milk', 'o...",frozenset({'bottled water'}),0.003336,0.213754,0.002566,0.769231,3.598670,0.001853,3.407065
24250,"frozenset({'sausage', 'soda', 'whole milk', 'o...",frozenset({'rolls/buns'}),0.003336,0.349756,0.002566,0.769231,2.199334,0.001399,2.817723


In [69]:
def fix_frozen_set(x): # takes panda dataframe and reformats antecedents and consequents as Frozensets for correct querying
    beginning = x.find("'") + 1
    end = x.rfind("'")
    data = x[beginning: end]
    data_remove_apostrophes = data.replace("'",'')
    new_data = data_remove_apostrophes.split(',')
    new_data_stripped = []
    for i in range(len(new_data)):
        new_data_stripped.append(copy.copy(new_data[i].strip()))
    if len(new_data_stripped) == 1:
        return frozenset(new_data_stripped)
    else:
        return frozenset([x for x in new_data_stripped])

In [70]:
df['antecedents'] = df['antecedents'].apply(fix_frozen_set)
df['consequents'] = df['consequents'].apply(fix_frozen_set)

In [71]:
df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24247,"(sausage, soda, whole milk, other vegetables, ...",(root vegetables),0.003849,0.230690,0.002566,0.666667,2.889878,0.001678,2.307929
24248,"(sausage, whole milk, other vegetables, rolls/...",(soda),0.003336,0.313318,0.002566,0.769231,2.455112,0.001521,2.975622
24249,"(sausage, soda, whole milk, other vegetables, ...",(bottled water),0.003336,0.213754,0.002566,0.769231,3.598670,0.001853,3.407065
24250,"(sausage, soda, whole milk, other vegetables, ...",(rolls/buns),0.003336,0.349756,0.002566,0.769231,2.199334,0.001399,2.817723


In [72]:
df[df['antecedents'] == {'rolls/buns', 'Instant food products'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
132,"(Instant food products, rolls/buns)",(root vegetables),0.005389,0.230690,0.002823,0.523810,2.270618,0.001580,1.615550
133,"(Instant food products, rolls/buns)",(soda),0.005389,0.313318,0.003593,0.666667,2.127764,0.001904,2.060046


In [42]:
df['antecedent_len'] = df['antecedents'].apply(lambda x: len(x))
df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1


In [43]:
df.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
24689,"(sausage, soda, other vegetables, root vegetab...","(whole milk, rolls/buns)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069,5
24690,"(sausage, whole milk, other vegetables, root v...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124,5
24691,"(soda, sausage, rolls/buns, root vegetables, b...","(other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189,5
24692,"(sausage, whole milk, rolls/buns, root vegetab...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947,5
24693,"(soda, sausage, whole milk, root vegetables, b...","(other vegetables, rolls/buns)",0.005132,0.146780,0.002566,0.500000,3.406469,0.001813,1.706441,5


In [44]:
df['consequent_len'] = df['consequents'].apply(lambda x: len(x))
df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1


In [45]:
df.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
24689,"(sausage, soda, other vegetables, root vegetab...","(whole milk, rolls/buns)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069,5,2
24690,"(sausage, whole milk, other vegetables, root v...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124,5,2
24691,"(soda, sausage, rolls/buns, root vegetables, b...","(other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189,5,2
24692,"(sausage, whole milk, rolls/buns, root vegetab...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947,5,2
24693,"(soda, sausage, whole milk, root vegetables, b...","(other vegetables, rolls/buns)",0.005132,0.146780,0.002566,0.500000,3.406469,0.001813,1.706441,5,2


In [60]:
df['consequents'].value_counts()

(whole milk)                         10108
(other vegetables)                    5942
(rolls/buns)                          3672
(soda)                                1894
(yogurt)                              1519
(root vegetables)                      294
(sausage)                              212
(other vegetables, whole milk)         203
(bottled water)                        202
(tropical fruit)                       135
(whole milk, rolls/buns)                89
(shopping bags)                         64
(citrus fruit)                          42
(soda, whole milk)                      41
(whole milk, yogurt)                    38
(pip fruit)                             30
(pastry)                                28
(other vegetables, rolls/buns)          26
(canned beer)                           25
(bottled beer)                          18
(brown bread)                           13
(pork)                                  12
(soda, other vegetables)                12
(frankfurte

In [46]:
ant_select = df['antecedent_len'] <= 2

In [48]:
df[ant_select]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1
...,...,...,...,...,...,...,...,...,...,...,...
15587,"(shopping bags, spread cheese)","(other vegetables, whole milk)",0.005132,0.191429,0.002566,0.500000,2.611930,0.001584,1.617141,2,2
15672,"(yogurt, spread cheese)","(other vegetables, whole milk)",0.008725,0.191429,0.004362,0.500000,2.611930,0.002692,1.617141,2,2
16093,"(sausage, soft cheese)","(whole milk, rolls/buns)",0.006672,0.178599,0.003849,0.576923,3.230272,0.002658,1.941493,2,2
16294,"(sausage, soft cheese)","(whole milk, yogurt)",0.006672,0.150372,0.003336,0.500000,3.325085,0.002333,1.699256,2,2


In [59]:
df[ant_select]['consequents'].value_counts()

(whole milk)                      2080
(other vegetables)                 929
(rolls/buns)                       505
(soda)                             217
(yogurt)                           113
(other vegetables, whole milk)      26
(tropical fruit)                    24
(root vegetables)                   19
(bottled water)                     16
(sausage)                           11
(whole milk, rolls/buns)             7
(citrus fruit)                       6
(pastry)                             5
(whole milk, yogurt)                 4
(other vegetables, rolls/buns)       3
(pip fruit)                          2
(whole milk, bottled water)          1
(soda, other vegetables)             1
(butter)                             1
(canned beer)                        1
(soda, whole milk)                   1
Name: consequents, dtype: int64

In [49]:
con_select = df['consequent_len'] == 1

In [50]:
df[con_select]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1
...,...,...,...,...,...,...,...,...,...,...,...
24683,"(sausage, soda, whole milk, other vegetables, ...",(root vegetables),0.003849,0.230690,0.002566,0.666667,2.889878,0.001678,2.307929,6,1
24684,"(sausage, whole milk, other vegetables, rolls/...",(soda),0.003336,0.313318,0.002566,0.769231,2.455112,0.001521,2.975622,6,1
24685,"(sausage, soda, whole milk, other vegetables, ...",(bottled water),0.003336,0.213754,0.002566,0.769231,3.598670,0.001853,3.407065,6,1
24686,"(sausage, soda, whole milk, other vegetables, ...",(rolls/buns),0.003336,0.349756,0.002566,0.769231,2.199334,0.001399,2.817723,6,1


In [61]:
df = df[con_select]
df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1
...,...,...,...,...,...,...,...,...,...,...,...
24683,"(sausage, soda, whole milk, other vegetables, ...",(root vegetables),0.003849,0.230690,0.002566,0.666667,2.889878,0.001678,2.307929,6,1
24684,"(sausage, whole milk, other vegetables, rolls/...",(soda),0.003336,0.313318,0.002566,0.769231,2.455112,0.001521,2.975622,6,1
24685,"(sausage, soda, whole milk, other vegetables, ...",(bottled water),0.003336,0.213754,0.002566,0.769231,3.598670,0.001853,3.407065,6,1
24686,"(sausage, soda, whole milk, other vegetables, ...",(rolls/buns),0.003336,0.349756,0.002566,0.769231,2.199334,0.001399,2.817723,6,1


In [62]:
df.drop(columns=['antecedent_len', 'consequent_len'], inplace=True)
df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [58]:
df[con_select]['consequents'].value_counts()

(whole milk)               10108
(other vegetables)          5942
(rolls/buns)                3672
(soda)                      1894
(yogurt)                    1519
(root vegetables)            294
(sausage)                    212
(bottled water)              202
(tropical fruit)             135
(shopping bags)               64
(citrus fruit)                42
(pip fruit)                   30
(pastry)                      28
(canned beer)                 25
(bottled beer)                18
(brown bread)                 13
(pork)                        12
(frankfurter)                 11
(whipped/sour cream)           9
(newspapers)                   8
(domestic eggs)                3
(white bread)                  3
(beef)                         2
(dessert)                      2
(butter)                       1
(curd)                         1
(fruit/vegetable juice)        1
(margarine)                    1
Name: consequents, dtype: int64

In [51]:
final_rule = (con_select & ant_select)

In [52]:
df.loc[ ~final_rule]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
3929,"(UHT-milk, other vegetables, beef)",(whole milk),0.004619,0.458045,0.003593,0.777778,1.698039,0.001477,2.438799,3,1
3930,"(UHT-milk, beef, whole milk)",(other vegetables),0.006928,0.376700,0.003593,0.518519,1.376476,0.000983,1.294546,3,1
3931,"(UHT-milk, beef, pip fruit)",(whole milk),0.003593,0.458045,0.002823,0.785714,1.715366,0.001177,2.529125,3,1
3932,"(UHT-milk, beef, rolls/buns)",(whole milk),0.004619,0.458045,0.003079,0.666667,1.455462,0.000964,1.625866,3,1
3933,"(UHT-milk, bottled beer, bottled water)",(other vegetables),0.003849,0.376700,0.002566,0.666667,1.769755,0.001116,1.869900,3,1
...,...,...,...,...,...,...,...,...,...,...,...
24689,"(sausage, soda, other vegetables, root vegetab...","(whole milk, rolls/buns)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069,5,2
24690,"(sausage, whole milk, other vegetables, root v...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124,5,2
24691,"(soda, sausage, rolls/buns, root vegetables, b...","(other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189,5,2
24692,"(sausage, whole milk, rolls/buns, root vegetab...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947,5,2


In [54]:
df1 = df.loc[final_rule]

In [55]:
df1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1


In [56]:
df1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722,1,1
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287,1,1
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529,1,1
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488,1,1
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121,1,1
...,...,...,...,...,...,...,...,...,...,...,...
3924,"(white wine, whipped/sour cream)",(whole milk),0.007442,0.458045,0.003849,0.517241,1.129238,0.000441,1.122622,2,1
3925,"(yogurt, whipped/sour cream)",(whole milk),0.047986,0.458045,0.027714,0.577540,1.260882,0.005734,1.282856,2,1
3926,"(white bread, yogurt)",(whole milk),0.028483,0.458045,0.016423,0.576577,1.258778,0.003376,1.279937,2,1
3927,"(white wine, yogurt)",(whole milk),0.016423,0.458045,0.009751,0.593750,1.296271,0.002229,1.334044,2,1


In [57]:
df1['consequents'].value_counts()

(whole milk)          2080
(other vegetables)     929
(rolls/buns)           505
(soda)                 217
(yogurt)               113
(tropical fruit)        24
(root vegetables)       19
(bottled water)         16
(sausage)               11
(citrus fruit)           6
(pastry)                 5
(pip fruit)              2
(canned beer)            1
(butter)                 1
Name: consequents, dtype: int64

In [63]:
df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [64]:
df.to_csv('Data/master_rules.csv', index=False)

In [65]:
print("DONE")

DONE
